In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

from utils import llava_load_model, llava_process_image, llava_generate
from lens_utils import llava_logit_lens

#### Paths

In [ ]:
data_path = "/home/drdo/vlm-compositionality/data"
dataset_folder = data_path+"/raw/sugarcrepe",
image_folder = data_path+"/raw/coco_val_2017"
model_name = "llava-hf/llava-1.5-13b-hf"
image_file = image_folder+"/000000008690.jpg"

#### Get hidden states

In [ ]:
# load image
# TODO: random image
image = Image.open(image_file)

# load model, processor
model, processor = llava_load_model(model_name)

# process image and prompt(default)
inputs = llava_process_image(image, processor, device=model.device)

# generate
outputs = llava_generate(inputs, model)

#### Get logit lens

In [ ]:
# TODO: norm before unembedding
# vocab_dim, num_layers, num_tokens
softmax_probs = llava_logit_lens(inputs, model, outputs)